In [1]:
from general_tools.notebook import gpu_utils
GPU = 2
gpu_utils.setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import glob
import matplotlib.pylab as plt

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf


from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, match_incomplete_to_complete_data
from tf_lab.autopredictors.scripts.virt_scan_data import plotting_default_params


from general_tools.notebook.tf import reset_tf_graph                                                  
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k, indices_in_iterable


from geo_tool import Point_Cloud

from tf_lab.nips.shape_net import pc_loader as sn_pc_loader

from tf_lab.autopredictors.exploration import latent_embedding_of_entire_dataset

from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs

/orions4-zfs/projects/lins2/Jingwei_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [6]:
'all_snc_rotated_conv_arch_2048pts_chamfer'
print experiment_name

top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
train_dir = osp.join(top_data_dir, 'OUT/models/nips/vanilla_ae/')
train_dir = osp.join(train_dir, experiment_name)

1_three_fc_dec_with_spn_2048pts_chamfer


In [7]:
conf = Conf.load(osp.join(train_dir, 'configuration'))
print conf
conf.n_output = conf.n_input

                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'non_linearity': <function relu at 0x7fdf05d7b320>, 'layer_sizes': [512, 1024, 6144]}
                       encoder: encoder_with_convs_and_symmetry
                  encoder_args: {}
               experiment_name: 1_three_fc_dec_with_spn_2048pts_chamfer
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 10
                     train_dir: /orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/vanilla_ae/1_three_fc_dec_with_spn_20

In [8]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

In [15]:
saved_epochs = read_saved_epochs(conf.train_dir)

In [16]:
last_epoch = saved_epochs[-1]
ae.restore_model(conf.train_dir, last_epoch, verbose=True)

Model restored in epoch 1000.


# top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
full_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
train_dir = osp.join(top_data_dir, 'OUT/models/nips/vanilla_ae/')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

In [13]:
full_file_names = pio.load_filenames_of_input_data(full_pclouds_path, '.ply')
full_pclouds, full_model_names, full_syn_ids = pio.load_crude_point_clouds(file_names=full_file_names,\
                                                                           n_threads=50, loader=sn_pc_loader)
print '%d files containing complete point clouds were found.' % (len(full_pclouds), )

array([[[  1.55409873e-01,  -1.95148841e-01,  -4.17097002e-01],
        [ -1.24767587e-01,  -1.94777861e-01,  -4.17097002e-01],
        [ -1.23644404e-01,  -1.91788822e-01,  -4.17097002e-01],
        ..., 
        [ -1.45030037e-01,  -2.74109747e-02,   4.14879888e-01],
        [ -1.33941725e-01,  -2.53829565e-02,   4.15186822e-01],
        [ -1.68545783e-01,   1.31114684e-02,   4.15194035e-01]],

       [[  1.52039647e-01,  -1.56266913e-01,  -4.39168990e-01],
        [  1.29050985e-01,  -1.53674230e-01,  -4.39168990e-01],
        [ -1.48278281e-01,  -1.50924847e-01,  -4.39168990e-01],
        ..., 
        [ -1.35444418e-01,   1.28694996e-02,   4.38588619e-01],
        [ -1.56956941e-01,   7.24414270e-03,   4.38841820e-01],
        [ -1.18339039e-01,   3.23241809e-04,   4.38997537e-01]],

       [[  1.76675156e-01,  -1.78966939e-01,  -3.95767123e-01],
        [ -1.43649906e-01,  -1.71165943e-01,  -3.95767123e-01],
        [  1.91921726e-01,  -1.68778956e-01,  -3.95767123e-01],
        

In [ ]:
ae.interpolate()